## 라이브러리 호출

In [1]:
import re   #정규식
import random   #데이터 증강용 난수
import numpy as np   #행렬 연산
import pandas as pd   #데이터프레임
import tensorflow as tf   #신경망
import matplotlib.pyplot as plt   #데이터 시각화
from tqdm.notebook import tqdm   #학습 과정 출력


import gensim   #Word2Vec
from konlpy.tag import Mecab   #형태소 분석


from nltk.translate.bleu_score import sentence_bleu   #bleu 연산
from nltk.translate.bleu_score import SmoothingFunction   #bleu 연산

## 데이터셋 불러오기(data augmentation한것과 챗봇데이터2 합치기)

In [2]:
dataset = pd.read_csv("ChatbotData2.csv")
#dataset.drop(("label"), axis="columns", inplace=True)
augmentation_data = pd.read_csv("augmentation.csv", encoding='utf-8-sig')
combined_data = pd.concat([dataset, augmentation_data], ignore_index=True)

In [3]:
len(combined_data)

47292

### 데이터 정제
    - 결측치 확인

In [4]:
combined_data.isnull().sum()

Unnamed: 0    0
Q             0
A             1
dtype: int64

In [5]:
## 데이터 샘플 확인
for i in range(10):
    print('Q :', combined_data['Q'][i])
    print('A :', combined_data['A'][i])
    print('----------------------')

Q : 12시 땡!
A : 하루가 또 가네요.
----------------------
Q : 1지망 학교 떨어졌어
A : 위로해 드립니다.
----------------------
Q : 3박4일 놀러가고 싶다
A : 여행은 언제나 좋죠.
----------------------
Q : 3박4일 정도 놀러가고 싶다
A : 여행은 언제나 좋죠.
----------------------
Q : PPL 심하네
A : 눈살이 찌푸려지죠.
----------------------
Q : SD카드 망가졌어
A : 다시 새로 사는 게 마음 편해요.
----------------------
Q : SD카드 안돼
A : 다시 새로 사는 게 마음 편해요.
----------------------
Q : SNS 맞팔 왜 안하지ㅠㅠ
A : 잘 모르고 있을 수도 있어요.
----------------------
Q : SNS 시간낭비인 거 아는데 매일 하는 중
A : 시간을 정하고 해보세요.
----------------------
Q : SNS 시간낭비인데 자꾸 보게됨
A : 시간을 정하고 해보세요.
----------------------


## 중복 데이터 체크

In [6]:
combined_data[combined_data.duplicated(['Q'])]

,Unnamed: 0,Q,A
196,196,고양이 키우고 싶어,가족들과 상의해보세요.
235,235,공시 준비 힘들어,잘 될 거예요.
1294,1294,돈 벌고 싶어,많이 벌수록 좋아요.
1445,1445,로또 번호 알려줘,알면 제가 하죠.
1481,1481,마음이 울적해,거리를 걸어보세요.
...,...,...,...
47287,35464,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!
47288,35465,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.
47289,35466,흑기사 해주는 짝남.,설렜겠어요.
47290,35467,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.


In [7]:
combined_data[combined_data['Q']==combined_data['Q'][11819]]

,Unnamed: 0,Q,A
11818,11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!
11819,11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.
23641,11818,훔쳐보는 것도 눈치 보임.,티가 나서 안 보이는 거죠!
23642,11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.
47287,35464,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!
47288,35465,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.


In [8]:
## 중복제거
combined_data[combined_data.duplicated(['A'])]

,Unnamed: 0,Q,A
3,3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
6,6,SD카드 안돼,다시 새로 사는 게 마음 편해요.
9,9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.
12,12,가끔 뭐하는지 궁금해,그 사람도 그럴 거예요.
18,18,가스불 켜놓고 나온거 같아,빨리 집에 돌아가서 끄고 나오세요.
...,...,...,...
47287,35464,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!
47288,35465,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.
47289,35466,흑기사 해주는 짝남.,설렜겠어요.
47290,35467,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.


In [9]:
combined_data[combined_data['A']==combined_data['A'][3]]

,Unnamed: 0,Q,A
2,2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
11825,2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
11826,3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.


In [10]:
combined_data[combined_data['A']==combined_data['A'][6]]

,Unnamed: 0,Q,A
5,5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.
6,6,SD카드 안돼,다시 새로 사는 게 마음 편해요.


## 중복 데이터 제거

In [11]:
combined_data.drop_duplicates(['A'], inplace=True)
len(dataset)

11823

In [12]:
combined_data.drop_duplicates(['Q'], inplace=True)
len(dataset)

11823

In [13]:
print("데이터 총 갯수:",len(dataset))

데이터 총 갯수: 11823


## 데이터 정제
    - 1. 영문 소문자 변환
    - 2. 알파벳, 문장부호, 한글, 숫자 제외는 제거
    - 3. 문장부호 양 여에 공백 추가
    - 4. 불필요한 공백 제거

In [14]:
def preprocess_sentence(sentence):
    if isinstance(sentence, str):
        sentence = sentence.lower().strip()
        sentence = re.sub(r'[" "]+', " ", sentence)
        sentence = re.sub(r"[^a-zA-Z?.!가-힣ㄱ-ㅎㅏ-ㅣ0-9]+", " ", sentence)
        sentence = sentence.strip()
    else:
        sentence = ""
    return sentence

In [15]:
questions = [preprocess_sentence(q) for q in combined_data['Q']]
answers = [preprocess_sentence(a) for a in combined_data['A']]

    
print('questions data size :', len(questions))
print('answers data size :', len(answers))
print()
for i in range(10):
    print('Q :', questions[i])
    print('A :', answers[i])    
    print("-------------------------")

questions data size : 20171
answers data size : 20171

Q : 12시 땡!
A : 하루가 또 가네요.
-------------------------
Q : 1지망 학교 떨어졌어
A : 위로해 드립니다.
-------------------------
Q : 3박4일 놀러가고 싶다
A : 여행은 언제나 좋죠.
-------------------------
Q : ppl 심하네
A : 눈살이 찌푸려지죠.
-------------------------
Q : sd카드 망가졌어
A : 다시 새로 사는 게 마음 편해요.
-------------------------
Q : sns 맞팔 왜 안하지ㅠㅠ
A : 잘 모르고 있을 수도 있어요.
-------------------------
Q : sns 시간낭비인 거 아는데 매일 하는 중
A : 시간을 정하고 해보세요.
-------------------------
Q : sns보면 나만 빼고 다 행복해보여
A : 자랑하는 자리니까요.
-------------------------
Q : 가끔 궁금해
A : 그 사람도 그럴 거예요.
-------------------------
Q : 가끔은 혼자인게 좋다
A : 혼자를 즐기세요.
-------------------------


## 챗봇 답변 데이터 start, end 붙이기 

In [16]:
def add_end_tok(sentences):
    new_sentences = []
    for sentence in sentences:
        # 문장을 단어 리스트로 변환
        tokenized_sentence = sentence.split()  # 공백 기준으로 토큰화
        # <start>와 <end> 토큰을 추가
        new_sentence = ["<start>"] + tokenized_sentence + ["<end>"]
        new_sentences.append(new_sentence)
    return new_sentences

In [17]:
# answers 리스트에 add_end_tok 함수 적용
tokenized_answers = add_end_tok(answers)

In [18]:
# 결과 확인
for i in range(5):
    print(tokenized_answers[i])

['<start>', '하루가', '또', '가네요.', '<end>']
['<start>', '위로해', '드립니다.', '<end>']
['<start>', '여행은', '언제나', '좋죠.', '<end>']
['<start>', '눈살이', '찌푸려지죠.', '<end>']
['<start>', '다시', '새로', '사는', '게', '마음', '편해요.', '<end>']


In [19]:
mecab = Mecab()
sen = "나는 오늘 저녁에 고기를 구워먹습니다."
morphed_sentence = mecab.morphs(sen)
print(morphed_sentence)

['나', '는', '오늘', '저녁', '에', '고기', '를', '구워', '먹', '습니다', '.']


## 데이터 토큰화
    - Mecab 이용

In [20]:
from konlpy.tag import Mecab
from konlpy.tag import Okt
# tokenize 함수 정의
mecab = Mecab()
def tokenize(corpus, vocab_size=50000):
    # Ensure all items in the corpus are strings and handle NaN values
    corpus = [str(sen) if isinstance(sen, str) else "" for sen in corpus]
    
    # Perform morphological analysis
    morph = [" ".join(mecab.morphs(sen)) for sen in corpus]
    
    # Initialize the tokenizer
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', num_words=vocab_size)
    tokenizer.fit_on_texts(morph)
    
    # Convert texts to sequences and apply padding
    tensor = tokenizer.texts_to_sequences(morph)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    return tensor, tokenizer, morph

In [21]:
questions_tensor, questions_tokenizer, questions_morph = tokenize(combined_data['Q'])
answers_tensor, answers_tokenizer, answers_morph = tokenize(combined_data['A'])

## 데이터 분리

In [22]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(questions_tensor, answers_tensor, test_size=0.05)

print("enc_train :", len(enc_train), "enc_val :", len(enc_val))
print("dec_train :", len(dec_train), "dec_val :",len(dec_val))

enc_train : 19162 enc_val : 1009
dec_train : 19162 dec_val : 1009


In [23]:
BATCH_SIZE = 64
train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).batch(batch_size=BATCH_SIZE)

# 모델 설계

In [24]:
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, (2*(i//2)) / np.float32(d_model))

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table

In [25]:
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_lookahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_enc_mask = generate_padding_mask(src)

    dec_lookahead_mask = generate_lookahead_mask(tgt.shape[1])
    dec_tgt_padding_mask = generate_padding_mask(tgt)
    dec_mask = tf.maximum(dec_tgt_padding_mask, dec_lookahead_mask)

    return enc_mask, dec_enc_mask, dec_mask

In [26]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions
        

    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x

    
    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)
        
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)
                        
        out = self.combine_heads(out)
        out = self.linear(out)
            
        return out, attention_weights

In [27]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
            
        return out

In [28]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        '''
        Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual
        
        return out, enc_attn

In [29]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        '''
        Masked Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        '''
        Multi-Head Attention
        '''
        residual = out
        out = self.norm_2(out)
        # Q, K, V 순서에 주의하세요!
        out, dec_enc_attn = self.enc_dec_attn(Q=out, K=enc_out, V=enc_out, mask=dec_enc_mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn

In [30]:
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]
    
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        out = x
    
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
        
        return out, enc_attns

In [31]:
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]
                            
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        out = x
    
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, dec_enc_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

In [32]:
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out

        
    def call(self, enc_in, dec_in, enc_mask, dec_enc_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        
        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, dec_enc_mask, dec_mask)
        
        logits = self.fc(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns

In [34]:
VOCAB_SIZE = 20000
transformer = Transformer(
    n_layers=1 ,  # 2
    d_model=368, # 512
    n_heads=8,
    d_ff=1024, # 1024
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=0.2,
    shared_fc=True,
    shared_emb=True)
		
d_model = 512

In [35]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=1000):
        super(LearningRateScheduler, self).__init__()
        
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

In [36]:
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)

In [37]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [38]:
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

In [39]:
EPOCHS = 10

for epoch in range(EPOCHS):
    total_loss = 0
    
    dataset_count = tf.data.experimental.cardinality(train_dataset).numpy()
    tqdm_bar = tqdm(total=dataset_count, desc=f"Epoch {epoch+1}/{EPOCHS}")
    
    for step, batch in enumerate(train_dataset):
        loss, enc_attns, dec_attns, dec_enc_attns = train_step(batch[0],batch[1],transformer,optimizer)
        total_loss += loss
                # tqdm 바 업데이트
        tqdm_bar.update(1)
        tqdm_bar.set_postfix({'loss': f'{total_loss / (step + 1):.4f}'})

    
    tqdm_bar.close()
    print(f"Epoch {epoch+1}/{EPOCHS} finished with loss: {total_loss / dataset_count:.4f}")

Epoch 1/10:   0%|          | 0/300 [00:00<?, ?it/s]

Epoch 1/10 finished with loss: 5.3144


Epoch 2/10:   0%|          | 0/300 [00:00<?, ?it/s]

Epoch 2/10 finished with loss: 2.9992


Epoch 3/10:   0%|          | 0/300 [00:00<?, ?it/s]

...

Epoch 8/10 finished with loss: 0.5557


Epoch 9/10:   0%|          | 0/300 [00:00<?, ?it/s]

Epoch 9/10 finished with loss: 0.4642


Epoch 10/10:   0%|          | 0/300 [00:00<?, ?it/s]

Epoch 10/10 finished with loss: 0.4014


# 추론

In [40]:
def sequences_to_texts(sequences, tokenizer):
    reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))  # 인덱스를 단어로 매핑
    return [" ".join([reverse_word_map.get(i, "") for i in seq]) for seq in sequences]


def generate(tokens, model, src_tokenizer, tgt_tokenizer):
    # 문장이 들어온다 str타입 1개가 들어와야한다
    padded_tokens = tf.keras.preprocessing.sequence.pad_sequences(tokens,
                                                           maxlen=MAX_LEN,
                                                           padding='post')
    
    ids = []
    output = tf.expand_dims([tgt_tokenizer.word_index['<start>']], 0)  
    for i in range(MAX_LEN):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(padded_tokens, output)

        predictions, _, _, _ = model(padded_tokens, 
                                      output,
                                      enc_padding_mask,
                                      combined_mask,
                                      dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if tgt_tokenizer.word_index['<end>'] == predicted_id:
            result = sequences_to_texts(ids, tgt_tokenizer)
            return result

        ids.append([predicted_id])
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)
    
    result = sequences_to_texts(ids, tgt_tokenizer)
    return result

In [41]:
def eval_bleu_single(model, src_sentence, tgt_sentence, src_tokenizer, tgt_tokenizer, verbose=True):
    src_tokens = src_tokenizer.texts_to_sequences(src_sentence)
    tgt_tokens = tgt_tokenizer.texts_to_sequences(tgt_sentence)

    if (len(src_tokens) > MAX_LEN): return None
    if (len(tgt_tokens) > MAX_LEN): return None

    reference = tgt_sentence
    candidate = generate(src_tokens, model, src_tokenizer, tgt_tokenizer)

    score = sentence_bleu([reference], candidate,
                          smoothing_function=SmoothingFunction().method1)

    if verbose:
        print("Source Sentence: ", src_sentence)
        print("Model Prediction: ", candidate)
        print("Real: ", reference)
        print("Score: %lf\n" % score)
        
    return score

In [43]:
que_corpus = combined_data['Q'].tolist()
ans_corpus = combined_data['A'].tolist()
src_tokenizer = questions_tokenizer  
tgt_tokenizer = answers_tokenizer    

In [45]:
questions_tokenizer.oov_token = "<OOV>"
answers_tokenizer.oov_token = "<OOV>"

In [49]:
questions_tokenizer

## BLEU 채점

In [50]:
test_idx = 1505

MAX_LEN = 50
eval_bleu_single(transformer, 
                 que_corpus[test_idx], 
                 ans_corpus[test_idx], 
                 src_tokenizer, 
                 tgt_tokenizer)

Source Sentence:  식욕폭발
Model Prediction:  ['이', '준비', '가', '없', '겠', '죠', '.', '그래서', '서로', '를', '꼭', '확인', '하', '고', '싶', '모습', '이', '라는', '수', '있', '겠', '지만', '라는', '준비', '라는', '준비', '라는', '준비', '라는', '준비', '가', '이', '라는', '준비', '가', '없', '겠', '지만', '라는', '준비', '하', '고', '싶', '이', '라는', '준비', '라는', '준비', '라는', '준비']
Real:  맛있게 먹으면 영칼로리!
Score: 0.003668



0.00366753025392797

## BLEU 설명
    - 기계 번역과 사람이 번역한 참조 번역 간의 n-gram 일치도를 기반으로 유사도를 계산(N-gram은 연속된 n개의 단어 또는 문자를 의미하며, 일반적으로 BLEU스코어에서는 1-gram부터 4-gram까지 고려합니다. 예를 들어, "I love machine learning"이라는 문장에서 2-gram은 "i love", "love machine", "machine translation"이 된다.
    - BLEU 스코어는 이러한 n-gram의 일치도를 바탕으로 0부터 1사이의 값
    - 1에 가까울수록 기계 번역 결과가 참조 번역과 유사하다는 것을 의미
    